This challenge is designed to give you basic familiarity with Fault tolerant quantum computation (FTQC). The goal of this challenge is to write the simplest FT quantum computer. It is composed of 4 steps.

1. Implementing basic QEC circuits as Supermarq benchmarks and running them on the Cirq Simulator to verify correctness of implementation
2. Implementing fault tolerant (FT) operations using the [Steane code](https://en.wikipedia.org/wiki/Steane_code) implemented in step 1 as Supermarq benchmarks and running them on the Cirq Simulator to verify correctness of implementation
3. Creating a FTQC by defining a function that performs FT operations and error correction on Cirq circuits
4. Running the Supermarq benchmarks that are in the [main repository](https://github.com/Infleqtion/client-superstaq/tree/main/supermarq-benchmarks/supermarq/benchmarks) (Except for the bit and phase code) as logical algorithms on top of the fault tolerant operations that you defined in step 2 and running them on the Cirq Simulator to verify correctness of implementation.

In [142]:
!pip install supermarq -q


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [143]:
import collections

import cirq
from cirq.contrib.qasm_import import circuit_from_qasm
from qiskit.synthesis import generate_basic_approximations
from qiskit.transpiler.passes.synthesis import SolovayKitaev

import supermarq
from supermarq.benchmark import Benchmark
from supermarq.benchmarks.bit_code import BitCode
from supermarq.benchmarks.phase_code import PhaseCode
from supermarq.benchmarks.ghz import GHZ
from supermarq.benchmarks.hamiltonian_simulation import HamiltonianSimulation
from supermarq.benchmarks.mermin_bell import MerminBell
from supermarq.benchmarks.qaoa_fermionic_swap_proxy import QAOAFermionicSwapProxy
from supermarq.benchmarks.qaoa_vanilla_proxy import QAOAVanillaProxy
from supermarq.benchmarks.vqe_proxy import VQEProxy

In [153]:
# Utilities
def compile_to_clifford_t(circuit: cirq.Circuit) -> cirq.Circuit:
    """Compiles to the clifford + t gateset (H, S, CNOT, T)"""
    qiskit_circuit = supermarq.converters.cirq_to_qiskit(circuit)
    qiskit_circuit.remove_final_measurements()
    basis = ["h", "s", "t"]
    approx = generate_basic_approximations(basis, depth=3)

    skd = SolovayKitaev(recursion_degree=2, basic_approximations=approx)
 
    discretized = skd(qiskit_circuit)

    qasm = discretized.qasm()
 
    return circuit_from_qasm(qasm)


def results_to_counts(circuit: cirq.Circuit, results: cirq.Result) -> collections.Counter[str]:
    """Transforms cirq.Result in collection.Counter"""
    new_collections_counter: collections.Counter[str] = collections.Counter()
    keys = sorted(circuit.all_measurement_key_names())
    num_qubits_measured_per_key = [results.measurements[key].shape[1] for key in keys]
    histogram = results.multi_measurement_histogram(keys=keys)

    for old_keys in histogram:
        new_key = "".join(
                f"{old_key:>0{num_qubits_measured}b}"
                for old_key, num_qubits_measured in zip(old_keys, num_qubits_measured_per_key)
            )
        new_collections_counter[new_key] = histogram[old_keys]

    return new_collections_counter

# Part 1

Implement the following QEC benchmarks using [Cirq's classical control](https://quantumai.google/cirq/build/classical_control).

In [150]:
class BitCode(BitCode):
    def circuit(self) -> cirq.Circuit:
        pass

In [151]:
class PhaseCode(Benchmark):
    def circuit(self) -> cirq.Circuit:
        pass 

In [152]:
class ShorCode(Benchmark):
    def circuit(self) -> cirq.Circuit:
        pass
    def score(self) -> int:
        pass

In [74]:
class FiveQubitCode(Benchmark):
    def circuit(self) -> cirq.Circuit:
        pass
    def score(self) -> int:
        pass

In [75]:
class SteaneCode(Benchmark):
    def circuit(self) -> cirq.Circuit:
        pass
    def score(self) -> int:
        pass

Run these circuits on the Cirq simulator and report the results

# Part 2

Now that you're comfortable with implementing and running error correcting codes, it's time to implement and run FT circuits. First define your FT operations.

In [76]:
class FTStatePrep(Benchmark):
    """Prepares logical |0⟩ or |1⟩ states"""
    def circuit() -> cirq.Circuit:
        pass
    def score() -> int:
        pass    

In [77]:
class FTMeasurement(Benchmark):
    """FT measurement gate"""
    def circuit() -> cirq.Circuit:
        pass
    def score() -> int:
        pass

In [78]:
class FTH(Benchmark):
    """FT Hadamard gate"""
    def circuit() -> cirq.Circuit:
        pass
    def score() -> int:
        pass

In [79]:
class FTS(Benchmark):
    """FT S gate"""
    def circuit() -> cirq.Circuit:
        pass
    def score() -> int:
        pass

In [80]:
class FTT(Benchmark):
    """FT T gate"""
    def circuit() -> cirq.Circuit:
        pass
    def score() -> int:
        pass

Run these circuits on the Cirq simulator and report the results

# Part 3

Now that you have fault tolerant operations and error correcting codes, you can finally build a fault tolerant quantum computer.

In [148]:
def compile_to_logical(circuit: cirq.Circuit) -> cirq.Circuit:
    circuit = compile_to_clifford_t(circuit)
    # ft_state_prep_0_L for all qubits()
    for op in circuit.all_operations():
        pass
        # ft_op
        # ft_error_correct
    if isinstance(op.gate, cirq.MeasurementGate):
        pass
        # ft_measure    

In [133]:
def ftqc(circuits: list[cirq.Circuit], repetitions=100):
    results = []
    for circuit in circuits:
        logical_circuit = compile_to_logical(circuit)
        simulator = cirq.Simulator()
        result = simulator.run(circuit, repetitions=repetitions)
        result = results_to_counts(circuit, result)
        results.append(result)
    return results

# Part 4

Run these Supermarq circuits on your FTQC

In [134]:
ghz = GHZ(3)
counts = ftqc([ghz.circuit()])
score = ghz.score(counts[0])

In [135]:
hs = HamiltonianSimulation(2, 1, 1)
counts = ftqc([hs.circuit()])
score = hs.score(counts[0])

In [136]:
mb = MerminBell(3)
counts = ftqc([mb.circuit()])
score = mb.score(counts[0])

/home/victory/venvs/quantum_hackathon_venv/lib/python3.11/site-packages/qiskit/synthesis/discrete_basis/commutator_decompose.py:140: RuntimeWarning: invalid value encountered in divide
  to_vector = to_vector / np.linalg.norm(to_vector)
/home/victory/venvs/quantum_hackathon_venv/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/home/victory/venvs/quantum_hackathon_venv/lib/python3.11/site-packages/qiskit/synthesis/discrete_basis/commutator_decompose.py:98: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  decomposition_angle = fsolve(objective, angle)[0]


In [119]:
qaoa = QAOAFermionicSwapProxy(4)
counts = ftqc([qaoa.circuit()])
score = qaoa.score(counts[0])

In [120]:
qaoa = QAOAVanillaProxy(4)
counts = ftqc([qaoa.circuit()])
score = qaoa.score(counts[0])

In [138]:
vqe = VQEProxy(2)
counts = ftqc(vqe.circuit())
score = vqe.score(counts)